In [18]:
import pandas as pd
import numpy as np
from datetime import datetime, date
from sweref99 import projections
import folium
from folium import plugins
from selenium import webdriver
import matplotlib.pyplot as plt
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, clear_output

In [3]:
path='./data/Insatser till brand i skog och mark 2000-2020.xlsx'

In [4]:
#tm is used for the conversions of easting and northing to longitude and latitude
tm = projections.make_transverse_mercator("SWEREF_99_TM")

In [36]:
df = pd.read_excel(path)


In [37]:
sample = df.sample(50)

In [38]:
for column in df:
    if df[column].isnull().any():
       print('{0} has {1} null values'.format(column, df[column].isnull().sum()))

sweref99Norr has 20718 null values
sweref99Ost has 20717 null values


In [39]:
#Function that calculates number of missing data in column of dataframe and prints result.
def missing(df,column):
       x = len(df)
       if df[column].isnull().any():
           print('{0} has total of {1} null values'.format(column, df[column].isnull().sum()))
           print ('In the column {0}'.format(column), round(df[column].count()-1/x * 100, 3), '% of the cells have missing values')
 

In [40]:
#Missing values of column 'Gras'
missing(sample,'sweref99Norr')
missing(sample,'sweref99Ost')
sample.isnull().sum(axis = 0)

sweref99Norr has total of 20718 null values
In the column sweref99Norr 79588.999 % of the cells have missing values
sweref99Ost has total of 20717 null values
In the column sweref99Ost 79589.999 % of the cells have missing values


ar                                  0
datum                               0
tid                                 0
kommun                              0
kommunKortNamn                      0
verksamhetText                      0
sweref99Norr                    20718
sweref99Ost                     20717
BEJBbrandorsakText                  0
arealProduktivSkogsmark_m2          0
arealAnnanTradbevuxenMark_m2        0
arealMarkUtanTrad_m2                0
dtype: int64

In [41]:
sample['TotArea'] = sample['arealProduktivSkogsmark_m2']+sample['arealAnnanTradbevuxenMark_m2']+sample['arealMarkUtanTrad_m2']
sample['Acres'] = sample['TotArea']/4046.86

In [42]:
#Functions for converting easting and northing to latitudes and longitudes.
def toLat(E,N):
    lat, lon = tm.grid_to_geodetic(N,E)
    return lat
def toLon(E,N):
    lat, lon = tm.grid_to_geodetic(N,E)
    return lon

In [43]:
#Applying functions to create to new columns, Longitude and Latitude for reported fires.
sample['Latitude'] = sample.apply(lambda row: toLat(row['sweref99Ost'],row['sweref99Norr']),axis=1)
sample['Longitude'] = sample.apply(lambda row: toLon(row['sweref99Ost'],row['sweref99Norr']),axis=1)
sample

,ar,datum,tid,kommun,kommunKortNamn,verksamhetText,sweref99Norr,sweref99Ost,BEJBbrandorsakText,arealProduktivSkogsmark_m2,arealAnnanTradbevuxenMark_m2,arealMarkUtanTrad_m2,TotArea,Acres,Latitude,Longitude
0,2000,2000-01-01,00:07:00,1272,Bromölla,Verksamhet inte knuten till en byggnad,6856.0,-905921.0,Fyrverkeri eller pyroteknik,0,6,0,6,0.001483,0.060540,2.467187
1,2000,2000-01-01,01:04:00,1452,Tranemo,Verksamhet inte knuten till en byggnad,NaN,NaN,Okänd,0,0,5,5,0.001236,NaN,NaN
2,2000,2000-01-02,01:05:00,2283,Sollefteå,"Lantbruk, inte bostad",NaN,NaN,Annan,0,0,15000,15000,3.706577,NaN,NaN
3,2000,2000-01-02,19:35:00,180,Stockholm,Verksamhet inte knuten till en byggnad,NaN,NaN,Okänd,0,5,0,5,0.001236,NaN,NaN
4,2000,2000-01-03,12:37:00,180,Stockholm,Verksamhet inte knuten till en byggnad,NaN,NaN,Barns lek med eld,0,50,0,50,0.012355,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100302,2020,2020-12-29,01:52:00,180,Stockholm,Verksamhet inte knuten till en byggnad,6586007.0,664319.0,Avsiktlig brand,0,1,0,1,0.000247,59.380626,17.892668
100303,2020,2020-12-29,18:56:00,117,Österåker,Verksamhet inte knuten till en byggnad,6599203.0,687136.0,Avsiktlig brand,0,0,1,1,0.000247,59.489416,18.305140
100304,2020,2020-12-30,12:32:00,380,Uppsala,Verksamhet inte knuten till en byggnad,6637240.0,653312.0,Okänd,0,0,20,20,0.004942,59.844320,17.736298
100305,2020,2020-12-31,15:17:00,1384,Kungsbacka,Verksamhet inte knuten till en byggnad,6360762.0,327099.0,Annan,0,0,1,1,0.000247,57.356841,12.125728


In [44]:
#We need to group fire sizes here but what sizes should we use? What is a "big" fire and what is a "small" fire?
sample['Acres'].describe()

count    100307.000000
mean          1.790165
std         121.364119
min           0.000000
25%           0.001236
50%           0.007413
75%           0.074132
max       31646.758227
Name: Acres, dtype: float64

In [56]:
sample = sample[sample['Latitude'].notna()]
sample = sample[sample['Longitude'].notna()]


In [57]:
#Group fires by size
small_wildfires = folium.FeatureGroup(TotArea = '< 10000 Acres')
medium_wildfires = folium.FeatureGroup(TotArea = '10,000 - 50,000 Acres')
large_wildfires = folium.FeatureGroup(TotArea = '50,000 - 100,000 Acres')
xl_wildfires = folium.FeatureGroup(TotArea = '> 100,000 Acres')

In [58]:
#Function that takes the dataframe and returns a map. Circles on the map are colored based on the size of the fire.
def add_FireCircle(df, m):
    for i, v in df.iterrows():
        
        fire_size = float(v['Acres'])
        
        #When hoovering over the circle, the popup will show the fire name, year, cause, state and size.
        #More values can be added.
        popup = """
        Kommun : <b>%s</b><br>
        Size (Acres) : <b>%s</b><br>
        Cause : <b>%s</b><br>
        Year: <b>%s</b><br>
        """ % (v['kommunKortNamn'], v['Acres'], 
            v['BEJBbrandorsakText'], 
            v['ar'])
        
        
        if fire_size < 10000:
            folium.CircleMarker(location = [v['Latitude'], 
                                            v['Longitude']],
                            radius = np.log(fire_size) * 0.8,
                            weight = 0,
                            tooltip = popup,
                            color = '#ffeda0',
                            fill_color = '#ffeda0',
                            fill_opacity = 0.7,
                            fill = True).add_to(small_wildfires)
            
        if fire_size in range(10000, 50000):
            folium.CircleMarker(location = [v['Latitude'], 
                                            v['Longitude']],
                            radius = np.log(fire_size),
                            weight = 0,
                            tooltip = popup,
                            color = '#feb24c',
                            fill_color = '#feb24c',
                            fill_opacity = 0.7,
                            fill = True).add_to(medium_wildfires)
        
        if fire_size in range(50000, 100000):
            folium.CircleMarker(location = [v['Latitude'], 
                                            v['Longitude']],
                            radius = np.log(fire_size) * 1.5,
                            weight = 0,
                            tooltip = popup,
                            color = '#fc4e2a',
                            fill_color = '#fc4e2a',
                            fill_opacity = 0.7,
                            fill = True).add_to(large_wildfires)

        if fire_size > 100000:
            folium.CircleMarker(location = [v['Latitude'], 
                                            v['Longitude']],
                            radius = np.log(fire_size) * 2,
                            weight = 0,
                            tooltip = popup,
                            color = '##b10026',
                            fill_color = '#b10026',
                            fill_opacity = 0.7,
                            fill = True).add_to(xl_wildfires)

    small_wildfires.add_to(m)
    medium_wildfires.add_to(m)
    large_wildfires.add_to(m)
    xl_wildfires.add_to(m)
    folium.LayerControl(collapsed = False).add_to(m)

    return m

In [59]:
#Map instanciation, with start point at the center of the US. Type of map is set to Stamen Terrain.
map = folium.Map(location = [59.334591, 18.063240],
               tiles = 'Stamen Terrain',
               zoom_start = 5.5)
map

In [60]:
m_sweden = add_FireCircle(sample, map)
m_sweden.save('sweden.html')

C:\Users\Johan\AppData\Local\Temp/ipykernel_29360/1013907757.py:22: RuntimeWarning: divide by zero encountered in log
  radius = np.log(fire_size) * 0.8,
